## Building A Chatbot
In this project i will design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.



In [59]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [60]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000148FC138CD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000148FC139840>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [61]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Deep and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Deep! That's a fascinating role. \n\nWhat kind of AI projects are you currently working on?  I'm always eager to learn about the latest developments in the field. \n\nIs there anything specific you'd like to discuss or ask me about? I can help with:\n\n* **Generating different creative text formats:**\n\nPoems, code, scripts, musical pieces, email, letters, etc.\n* **Answering your questions in an informative way:** Even if they are open ended, challenging, or strange.\n* **Summarizing factual topics:** Provide concise summaries of factual topics.\n\nLet me know how I can be of service!\n", response_metadata={'token_usage': {'completion_tokens': 143, 'prompt_tokens': 22, 'total_tokens': 165, 'completion_time': 0.26, 'prompt_time': 0.000138149, 'queue_time': 0.013281292, 'total_time': 0.260138149}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3aeecdbf-d351-4058-9fa4-57623ab99

In [62]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Deep and I am a Chief AI Engineer"),
        AIMessage(content="Hello Deep! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Deep and that you are a Chief AI Engineer!  😄 \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat. \n", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 97, 'total_tokens': 145, 'completion_time': 0.087272727, 'prompt_time': 0.003446713, 'queue_time': 0.011144755, 'total_time': 0.09071944}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd5741f3-cc0e-45c3-9709-48aacc46bc5d-0', usage_metadata={'input_tokens': 97, 'output_tokens': 48, 'total_tokens': 145})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [63]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [64]:
config={"configurable":{"session_id":"chat1"}}

In [65]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Deep and I am a Chief AI Engineer")],
    config=config
)

In [66]:
response.content

"Hi Deep, it's nice to meet you! \n\nThat's a very impressive title. What kind of work are you doing as a Chief AI Engineer?  I'm always interested in learning more about how AI is being used in different fields.  \n\nPerhaps you could tell me about:\n\n* **The industry you work in:**  What kind of company do you work for?\n* **Some of your current projects:** What are you most excited about working on right now?\n* **The biggest challenges you face:** What are some of the difficulties of being a Chief AI Engineer?\n\n\nI'm eager to hear more about your experiences!\n"

In [67]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Deep.  😄  \n\nI remember that from our introduction!  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 174, 'total_tokens': 205, 'completion_time': 0.056363636, 'prompt_time': 0.006677036, 'queue_time': 0.007005023, 'total_time': 0.063040672}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2d67db67-416d-4e04-a86d-870f5fd9475c-0', usage_metadata={'input_tokens': 174, 'output_tokens': 31, 'total_tokens': 205})

In [68]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name.  If you'd like to tell me your name, I'd be happy to use it! 🙂  \n\n"

In [69]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nIs there anything I can help you with today? 😊  \n"

In [70]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"You're John!  \n\nI remember. 😊  \n\nIs there anything else I can help you with, John?"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [71]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [72]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Deep")]})

AIMessage(content="It's nice to meet you, Deep! What can I do for you today? 😊  \n\nI'm here to answer your questions and help with any tasks you might have. Just ask! \n", response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 31, 'total_tokens': 76, 'completion_time': 0.081818182, 'prompt_time': 0.001825619, 'queue_time': 0.011311639, 'total_time': 0.083643801}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c455a77a-cca0-4e01-b381-96bd6eb55495-0', usage_metadata={'input_tokens': 31, 'output_tokens': 45, 'total_tokens': 76})

In [73]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [74]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Deep")],
    config=config
)

response

AIMessage(content="Hi Deep! It's nice to meet you.  \n\nHow can I help you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 31, 'total_tokens': 56, 'completion_time': 0.045454545, 'prompt_time': 0.001129967, 'queue_time': 0.014168163000000001, 'total_time': 0.046584512}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ad2ccc5-07d8-4cc7-ae08-5cd7414eced7-0', usage_metadata={'input_tokens': 31, 'output_tokens': 25, 'total_tokens': 56})

In [75]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Deep! You told me at the beginning of our conversation.  😊  \n'

In [76]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [77]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Deep")],"language":"German"})
response.content

'Hallo Deep! \n\nFreut mich, dich kennenzulernen. 🙂  Wie kann ich dir helfen? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [78]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [79]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Deep")],"language":"German"},
    config=config
)
repsonse.content

'Hallo Deep, schön dich kennenzulernen! \n\nWas kann ich für dich tun? 😊\n'

In [80]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "German"},
    config=config,
)

In [81]:
response.content

'Dein Name ist Deep, hast du mir gerade gesagt.  Ist da etwas, das ich noch für dich tun kann? \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [82]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [83]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have taste buds or personal preferences like liking ice cream!  \n\nWhat's your favorite flavor? 😊🍦  \n"

In [84]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: 2 + 2 \n\nIs there anything else I can help you with? 😊  \n'

In [85]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [86]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you, so I don't know your name.  \n\nIf you'd like to tell me, I'd love to know! 😊  \n\n"

In [87]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations.  \n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"